In [3]:

!pip install spylls
!pip install pyspellchecker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 41.7 MB/s eta 0:00:00


In [4]:
import re
from collections import Counter
import tensorflow as tf
from spellchecker import SpellChecker

# COMMON_WORDS жагсаалт
COMMON_WORDS = {
    "юм", "нь", "ба", "болон", "байна", "гэх", "бол",
    "гэж", "хүн", "хэн", "үг", "хэдий", "гэсэн", "байх",
    "хэрэг", "шиг", "дээр", "л", "ч"
}

def clean_text(text):
    # Remove punctuation and convert to lowercase
    text = re.sub(r"[^\u0400-\u04FF\s]", "", text)
    return text.lower()

def load_spell_checker():
    spell_checker = SpellChecker(language=None)
    return spell_checker

def check_spelling(spell_checker, word):
    """Check spelling and get suggestions"""
    suggestions = spell_checker.candidates(word)
    return suggestions if suggestions else []

def process_text(text):
    # Initialize spell checker
    spell_checker = load_spell_checker()

    # Clean and tokenize text
    cleaned_text = clean_text(text)
    words = cleaned_text.split()

    # Spelling check and word analysis
    misspelled = {}
    word_counts = Counter()

    for word in words:
        # Count words
        word_counts[word] += 1

        # Check spelling (only for words not in common words)
        if word not in COMMON_WORDS:
            suggestions = check_spelling(spell_checker, word)
            if suggestions:
                misspelled[word] = list(suggestions)

    # Filter out common words and get top words
    filtered_words = {word: count for word, count in word_counts.items() if word not in COMMON_WORDS}
    top_words = sorted(filtered_words.items(), key=lambda x: x[1], reverse=True)[:10]

    # Simple text classification (based on keywords)
    def classify_text(text):
        categories = [
            ("кино", "Кино сэдэв"),
            ("аав", "Гэр бүл"),
            ("охин", "Хүний харилцаа"),
            ("хайр", "Сэтгэл хөдлөл")
        ]

        for keyword, category in categories:
            if keyword in text.lower():
                return category
        return "Бусад"

    return {
        "misspelled": misspelled,
        "unique_words": len(set(words)),
        "total_words": len(words),
        "top_words": top_words,
        "classification": classify_text(text)
    }

# Review текст
text = """Заримдаа аав ээжээсээ илүү үр хүүхэд нь хариуцлагатай байх ёстой ч юм шиг..
Хувь заяа (review)
Хувьсал продакшны киног үзэхийн тулд унд хоол, нус нулимсны алчуураа сайн бэлдэх хэрэгтэй. Учир нь тэд хэзээ ч минутандаа харам байдаггүй. Бас чамайг уйлуулж, ухааруулах гэж л кино туурвидаг.
"Хувь Заяа" кино нь эхийн хайраар дутсан өсвөр насны Хувьзаяа охины ээдрээт амьдралын талаар өгүүлэх драм кино бөгөөд найруулагч Г.Бямбасүрэнгийн ээлжит бүтээл юм.
Тус киноны үйл явдал Говийн нэгэн жижиг суманд өрнөх бөгөөд аавтайгаа сэтгэл хангалуун амьдардаг Хувьзаяа охины гэрт нэгэн үл мэдэгдэх бүсгүй ирснээр ээдрээт асуудлууд ар араасаа хөврөнө.
Ээжгүй охиныхоо сэтгэлийг дүүрэн байлгах гэж бүхий л амьдралаа зориулсан аав (Эрдэнэзаан) ганцхан удаа ч гэсэн охиноо элэг бүтэн амьдруулах гэсэн хүсэл мөрөөдөл нь эцэстээ юунд хүргэхийг тус киноноос үзээрэй.
Жүжигчин Эрдэнэзаантай юу ярихав. "Гавьяат" цол тэмдгээ чимж яваа нэгэн шүү дээ. Энэ кинон дээрх охиныхоо төлөө дотроо уйлж, гаднаа инээсэн аавын дүр нь яг л өөрөө мэт үнэмшилтэй санагдав.
Элсэн цөлд болоод ч тэрүү "Хувь Заяа" киноны зураг авалт, өнцөг шийдлүүд нь тун ч шинэлэг. Өдрийн зураг авалт, шөнийн зураг авалт, цаашлаад элсэн шуурга, зэрэглээ гээд л энэ киноны зураг авалт тэр чигтээ masterpiece байв.
Мөн цөөхөн хэрнээ чанартай хэдхэн жүжигчин нь гол зуртал үнэн жүжиглэх аж. Кино эхлээд 5 минут ч болоогүй байтал манай эхнэр "Жүжигчин Урнаа шиг гое хөгшрөх юмсан" гэж хэлсэн юмдаг. Нээрээ л энэ хүн хөгшрөхгүй нь. Дарс удах тусам чанараа хадгалдаг гэдэг шиг гавьяат маань улам л цэмбийсээр. Мөн Хувьзаяа охины дүрд тоглосон энэ бяцхан жүжигчин хэн бэ? Энэ насан дээр ийм мундаг дүрээ бүтээдэг хэнийг ч харж байсангүй ээ.
Нөгөө саахалт айлын санаа нэг гэдэг үгийн утгыг уран яруугаар тайлан харуулах агуу жүжигчин Мэндбаяр гуайг дурдахгүй өнгөрч болохгүй байх. Түүний дүр хөдөө нутгийн эгэл бор амьдралтай ч эгэлгүй сэтгэлтэй Монголчуудыг нэвт шувт илтгэж байлаа.
Тус киноны зохиол, хэлэх гэсэн санаа нь хамгийн ихээр сэтгэлд хүрч байв. Бид ер нь яагаад эцэг эхээ бидний төлөө амьдарч үхэх ёстой гэж боддог юм бэ? Заримдаа тэд ч бас хэн нэгнээр хайрлагдаж, аз жаргалын хэлтэрхийнээс ч болов хүртэх эрхтэй гэдгийг хожом нь ухаарах ёстойм гэж үү?
Ойрд гоё гоё драм кино үзээд гоё байна өө. Би чинь бас уян зөөлөн хүн юмсанж ххэ.
Яахав зарим хүмүүсийн сэтгэгдлийг ажаад байхад Хувьсалын кинонууд хэтэрхий хар бараан гэлцэх нь олон. Бид үзмээр байгаа киноогоо сонгоод үзэх боломжтой цаг үед амьдардаг шүү гэж л хэлье даа."""

# Process the text
result = process_text(text)

# Print results
print("Spelling Analysis:")
print("----------------")
if result["misspelled"]:
    print("Misspelled Words:")
    for word, suggestions in result["misspelled"].items():
        print(f" - {word}: {', '.join(suggestions)}")
else:
    print("No misspelled words detected.")

print("\nWord Statistics:")
print("----------------")
print(f"Total Words: {result['total_words']}")
print(f"Unique Words: {result['unique_words']}")

print("\nTop 10 Frequent Words:")
print("--------------------")
for word, count in result['top_words']:
    print(f" - {word}: {count}")

print(f"\nText Classification: {result['classification']}")

Spelling Analysis:
----------------
Misspelled Words:
 - заримдаа: заримдаа
 - ээжээсээ: ээжээсээ
 - илүү: илүү
 - хүүхэд: хүүхэд
 - хариуцлагатай: хариуцлагатай
 - ёстой: ёстой
 - хувь: хувь
 - заяа: заяа
 - хувьсал: хувьсал
 - продакшны: продакшны
 - киног: киног
 - үзэхийн: үзэхийн
 - тулд: тулд
 - хоол: хоол
 - нулимсны: нулимсны
 - алчуураа: алчуураа
 - сайн: сайн
 - бэлдэх: бэлдэх
 - хэрэгтэй: хэрэгтэй
 - учир: учир
 - хэзээ: хэзээ
 - минутандаа: минутандаа
 - харам: харам
 - байдаггүй: байдаггүй
 - чамайг: чамайг
 - уйлуулж: уйлуулж
 - ухааруулах: ухааруулах
 - кино: кино
 - туурвидаг: туурвидаг
 - эхийн: эхийн
 - хайраар: хайраар
 - дутсан: дутсан
 - өсвөр: өсвөр
 - насны: насны
 - хувьзаяа: хувьзаяа
 - охины: охины
 - ээдрээт: ээдрээт
 - амьдралын: амьдралын
 - талаар: талаар
 - өгүүлэх: өгүүлэх
 - драм: драм
 - бөгөөд: бөгөөд
 - найруулагч: найруулагч
 - гбямбасүрэнгийн: гбямбасүрэнгийн
 - ээлжит: ээлжит
 - бүтээл: бүтээл
 - киноны: киноны
 - явдал: явдал
 - говийн: говийн
 -